# Import thư viện.

In [ ]:
# Tran Kim Thanh.
from pyspark.sql import * 
from pyspark.sql.functions import *

# Khởi tạo Spark Session.

In [ ]:
spark = SparkSession.builder \
    .appName("Analysis") \
    .getOrCreate()

# Đọc file dữ liệu và tổng quan.

In [ ]:
df = spark.read.csv('./data/preprocessed_data.csv', header=True, inferSchema=True, quote='"' , escape ='"', multiLine=True)

In [ ]:
df.show()

### Lấy số views cao nhất của mỗi video.

In [ ]:
windowSpec = Window.partitionBy("title").orderBy(col("views").desc())
windowSpec

In [ ]:
view_df = df.withColumn("view_rank", row_number().over(windowSpec))
view_df.show()

In [ ]:
view_df = view_df.filter(col('view_rank') == 1)
view_df = view_df.drop(view_df.view_rank)

In [ ]:
view_df = view_df.sort(desc('views'))

In [ ]:
view_df.show()

In [ ]:
view_df.select(sum('views')).show()

# Phân tích.

## Video có nhiều lượt xem nhất

In [ ]:
view = view_df.limit(20).toPandas() # Đoạn sau giá trị giảm chậm, không chênh nhiều.
ax = view.plot.bar(x = 'title', y='views', figsize = (20,9))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}')) 
ax.set_xlabel('Video Title')
ax.set_ylabel('Total views')

## Video có nhiều tương tác nhất.

In [ ]:
# Tương tác bằng tổng số likes + dislikes + comments.
view_df = view_df.withColumn('total_reactions', col('likes')+col('dislikes')+col('comment_count'))
highest_interaction = view_df.groupBy('title').agg(max('total_reactions').alias('total_reactions'))
highest_interaction = highest_interaction.orderBy(col("total_reactions").desc())

In [ ]:
highest_interaction.show()

#### Plot biểu đồ.

In [ ]:
import pyspark.pandas as ps
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter

In [ ]:
interaction_df = highest_interaction.limit(20).toPandas() # Đoạn sau giá trị giảm chậm, không chênh nhiều.
ax = interaction_df.plot.bar(x = 'title', y='total_reactions', figsize = (20,9))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}')) 
ax.set_xlabel('Video Title')
ax.set_ylabel('Total Reactions')

## Videos có nhiều lượt likes nhất.

In [ ]:
highest_like = view_df.groupBy('title').agg(max('likes').alias('likes'))
highest_like = highest_like.orderBy(col("likes").desc())

In [ ]:
highest_like.show()

#### Plot biểu đồ.

In [ ]:
pandas_total_like_df = highest_like.limit(20).toPandas()
ax = pandas_total_like_df.plot.bar(x = 'title', y='likes', figsize = (20,9))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}')) 
ax.set_xlabel('Video Title')
ax.set_ylabel('Number of Likes')

## Videos có nhiều lượt dislikes nhất.

In [ ]:
highest_dislike = view_df.groupBy('title').agg(max('dislikes').alias('dislikes'))
highest_dislike = highest_dislike.orderBy(col("dislikes").desc())

In [ ]:
highest_dislike.show()

#### Plot biểu đồ.

In [ ]:
pandas_total_dl_df = highest_dislike.limit(50).toPandas()
ax = pandas_total_dl_df.plot.bar(x = 'title', y='dislikes', figsize = (20,9))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}')) 
ax.set_xlabel('Video Title')
ax.set_ylabel('Number of Disikes')

## Videos có nhiều comments nhất.

In [ ]:
highest_cmt = view_df.groupBy('title').agg(max('comment_count').alias('comment_count'))
highest_cmt = highest_cmt.orderBy(col("comment_count").desc())

In [ ]:
highest_cmt.show()

#### Plot biểu đồ.

In [ ]:
pandas_total_cmt_df = highest_cmt.limit(20).toPandas()
ax = pandas_total_cmt_df.plot.bar(x = 'title', y='comment_count', figsize = (20, 9))
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x:,.0f}')) 
ax.set_xlabel('Video Title')
ax.set_ylabel('Number of Comments')